<a href="https://colab.research.google.com/github/mattiapocci/PhilosoperRank/blob/master/NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#***Imports and Drive Mount***

In [8]:

#!pip3 install -U spacy[cuda92]
#!python3 -m spacy download en_core_web_lg
#https://stackoverflow.com/questions/54334304/spacy-cant-find-model-en-core-web-sm-on-windows-10-and-python-3-5-3-anacon

import sys
import time
import os
import numpy as np
import random
import json
import spacy
from spacy import displacy
import nltk
import pprint
from nltk.tokenize import word_tokenize

nltk.download('punkt')

nlp = spacy.load("en_core_web_sm")
spacy.prefer_gpu()

from google.colab import drive

drive.mount('/content/drive')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# ***Language Analysis***

In [17]:

"""
1. X cite(s|d) Y1, Y2, …, Yn as (an) influence(s)
2. X was influenced by Y1, Y2, …, Yn
3. Y has been cited as an influence by X1, X2, …, Xn
4. Y influence on X1, X2, …, Xn …
5. Y1, Y2, …, Yn influenced hi(m|s) …


X's can be changed with his
Influence:
X be an influence (of/on/for/to) Y 
X's influence (of/on/for/to) Y
X's have influence on Y
X be Y's influence
Y cite X as influence
X was cited as influence by Y

influences:
influences (include/of) X
X be Y's influences
X and X were influences of Y

influencing:
none

influenced:
Y be influenced by X
X have influenced Y
X influenced Y
Y's thinking was influenced by X
X influenced Y's work
X's work influenced Y
"""

def get_info(sent_list):
  for sent in sent_list:
    print(sent)
    doc = nlp(sent)
    displacy.render(doc, style="dep", jupyter=True)  
    print("CHUNKS")
        
    for chunk in doc.noun_chunks:
        print(chunk.text, chunk.root.text, chunk.root.dep_, 
                chunk.root.head.text)
    print("TREE")
    for token in doc:
        print(token.text, token.dep_, token.head.text, token.head.pos_,
                [child for child in token.children])


def process(phrase):
    influenced = []
    influencers = []
    doc = nlp(phrase)
    compounds = []
    for token in doc:
        if token.dep_ == "nsubjpass" and token.head.text in ["influenced", "influences"] :
            influenced.append(token.text)
        if token.dep_ == "nsubj" and token.head.text in ["influenced", "influences"]:
            influencers.append(token.text)
        if token.dep_ == "pobj":
            flag = 0
            tok = token.text
            children = token.children
            while flag == 0:
                if token.head.dep_ == "pobj" or token.head.dep_ in ["prep", "agent","conj"]:
                    token = token.head
                elif token.head.text in ["influenced", "influences"]:
                    influencers.append(tok)
                    flag = 1
                else:
                    flag = 1
        if token.dep_ == "conj":
            flag = 0
            tok = token.text
            children = token.children
            while flag == 0:
                if token.head.dep_ == "pobj" or token.head.dep_ == "prep" or token.head.dep_ =="agent" or token.head.dep_ =="conj":
                    token = token.head
                elif token.head.text in ["influenced", "influences"]:
                    influencers.append(tok)
                    flag = 1
                else:
                    flag = 1
        if token.dep_ == "dobj":
            flag = 0
            tok = token.text
            children = token.children
            while flag == 0:
                if token.head.dep_ == "pobj" or token.head.dep_ == "dobj" or token.head.dep_ == "prep" or token.head.dep_ =="agent":
                    token = token.head
                elif token.head.text in ["influenced", "influences"]:
                    influencers.append(tok)
                    flag = 1
                else:
                    flag = 1
        
        if token.dep_ == "compound":
            compounds.append(token)
    
    for token in compounds:
        if token.head.text in influenced:
            influenced.append(token.text)
        if token.head.text in influencers:
            influencers.append(token.text)
    return influenced, influencers
    
phrases=["Mark was influenced by Jhon",
         "Jhon influenced Mark",
         "Mark was an influence of Jhon and Albert",
         "Jhon influenced him",
         "Adler was also influenced by the philosophies of Immanuel Kant, Friedrich Nietzsche, Rudolf Virchow and the statesman Jan Smuts ",
         "He is cited as an influence on John Milton",
         ]

phrases_influence=["He was an influence of her",
                   "Jhon was an influence of Mark",
                   "Milton was the main influencer of Mallio and Tullio",
                   "Jhon's influence on Mark",
                   "Markov's influence on Tito is the main reason of \"La Bella e la Bestia\", the main work of Tito's career. ",
                   "Jhon was Mark's influence",
                   "At the university she was introduced to the writings of Aristotle and Plato , who would be her greatest influence and counter-influence , respectively",
                   "Jhon had influence on Mark's work",
                   "Hegel had an importantr influence regarding all the different works of Einstein career",
                   "Mark cites Jhon as an influence",
                   "Hegel cited Shopenauer as the main influence of his work",
                   "Jhon was cited by mark as an influence",
                   "Adler is cited as an influence on John Milton"]
phrases_influences=["influences include him",
                   "influences include Jhon",
                   "influences of his work include Jhon Milton and the author of  \"La Casa\", Rick Morty",
                   "Jhon and Adam were Mark's influences",
                   "Aristotele, Anassagora and Pluto were Antichiola main influences",
                   "Jhon and Adam where influences of Mark",
                   "Macchiavelli and Dante Alighieri where the main influences of Boccaccio when he firstly thougth about Decameron"]
phrases_influenced=["He was influenced by him",
                   "Mark was influenced by Jhon",
                   "Adler was also influenced by the philosophies of Immanuel Kant, Friedrich Nietzsche, Rudolf Virchow and the statesman Jan Smuts",
                   "Jhon have influenced Mark",
                   "Macchiavelli with his work have deeply influenced the philosophies of Mathma Ghandi",
                   "Jhon influenced Mark",
                   "Anassagora influenced Pitagora on the ideas regarding geometry",
                   "Mark's work was influenced by Jhon",
                   "Gandhi's \"Power to the Man\" was influenced by the medioeval thinker Mark Robben",
                   "Jhon's work influenced Mark",
                   "Marconi's work on the electromagnetic field influenced Einsteins ideas of photon's transmission",
                   "Jhon influenced Mark's work",
                   "Pluto influenced the idea of time of Aristotele and Minos"]
phrases_inspire=["He seemed to inspire her",
                    "These revolutionizing ideas of Wang Yangming would later inspire prominent Japanese thinkers like Motoori Norinaga",
                   "John's work inspire Mark",]
phrases_inspiration=["John's inspiration include him",
                   "John was inspiration for Mark",
                   "Rozanov is the main source of inspiration for Dmitry Galkovsky ",
                   "Jhon and Adam became inspiration for Mark",
                   "Jhon's work provided inspiration for Mark",
                   "He got the inspiration for this text from Schleiermacher ’ s Über die Religion ",
                   "Jhon's work was inspiration for Mark",
                   "While Murdoch 's thought is an inspiration for Conradi",
                   "Jhon's work served as inspiration to Mark",
                   "Lucian 's True Story inspired Cyrano de Bergerac , whose writings later served as inspiration for Jules Verne ",
                   "Mark took inspiration from John",
                   "He also took inspiration from phenomenologist epistemology",
                   "Mark drew inspiration from John",
                   "In particular , he drew inspiration from a Chinese Buddhist master named Tao-cho",
                   "Mark provided inspiration to John"]
phrases_inspired=["He was inspired by him",
                   "Mark have inspired Jhon",
                   "Jhon had been inspired by Marks",
                   "In it , Petrarch claimed to have been inspired by Philip V of Macedon",
                   "Jhon's thinking was inspired by Mark",
                   "Newton 's work on infinite series was inspired by Simon Stevin 's decimals",
                   "Jhon's work was inspired by Marks",
                   "Schiller was inspired by the play Julius of Tarent by Johann Anton Leisewitz .",
                   "Mark was inspired by Jhon",
                   "As a youth , he was inspired by Mencius ’ proposition",
                   "Jhon inspired Mark",
                   "It also inspired him to take falsifiability as his criterion of demarcation between what is , and is not , genuinely scientific",
                   "Jhon inspired Mark's work",
                   "Spinoza inspired the poet Shelley to write his essay"]

for phr in phrases_influenced:
  get_info([phr])
  print(process(phr))


He was influenced by him


CHUNKS
He He nsubjpass influenced
him him pobj by
TREE
He nsubjpass influenced VERB []
was auxpass influenced VERB []
influenced ROOT influenced VERB [He, was, by]
by agent influenced VERB [him]
him pobj by ADP []
(['He'], ['him'])
Mark was influenced by Jhon


CHUNKS
Mark Mark nsubjpass influenced
Jhon Jhon pobj by
TREE
Mark nsubjpass influenced VERB []
was auxpass influenced VERB []
influenced ROOT influenced VERB [Mark, was, by]
by agent influenced VERB [Jhon]
Jhon pobj by ADP []
(['Mark'], ['Jhon'])
Adler was also influenced by the philosophies of Immanuel Kant, Friedrich Nietzsche, Rudolf Virchow and the statesman Jan Smuts


CHUNKS
Adler Adler nsubjpass influenced
the philosophies philosophies pobj by
Immanuel Kant Kant pobj of
Friedrich Nietzsche Nietzsche conj Kant
Rudolf Virchow Virchow conj Nietzsche
the statesman statesman conj Virchow
Jan Smuts Smuts appos statesman
TREE
Adler nsubjpass influenced VERB []
was auxpass influenced VERB []
also advmod influenced VERB []
influenced ROOT influenced VERB [Adler, was, also, by]
by agent influenced VERB [philosophies]
the det philosophies NOUN []
philosophies pobj by ADP [the, of]
of prep philosophies NOUN [Kant]
Immanuel compound Kant PROPN []
Kant pobj of ADP [Immanuel, ,, Nietzsche]
, punct Kant PROPN []
Friedrich compound Nietzsche PROPN []
Nietzsche conj Kant PROPN [Friedrich, ,, Virchow]
, punct Nietzsche PROPN []
Rudolf compound Virchow PROPN []
Virchow conj Nietzsche PROPN [Rudolf, and, statesman]
and cc Virchow PROPN []
the det statesman NOUN []
statesman conj Virchow PROPN [the, Smuts]
Jan compound Smuts PROPN []
Smuts appos statesman NOUN [Jan]
(['

CHUNKS
Jhon Jhon nsubj influenced
Mark Mark dobj influenced
TREE
Jhon nsubj influenced VERB []
have aux influenced VERB []
influenced ROOT influenced VERB [Jhon, have, Mark]
Mark dobj influenced VERB []
([], ['Jhon', 'Mark'])
Macchiavelli with his work have deeply influenced the philosophies of Mathma Ghandi


...

CHUNKS
Gandhi's "Power Power nsubjpass influenced
the Man Man pobj to
the medioeval thinker Mark Robben Robben pobj by
TREE
Gandhi poss Power NOUN ['s]
's case Gandhi PROPN []
" punct Power NOUN []
Power nsubjpass influenced VERB [Gandhi, ", to]
to prep Power NOUN [Man]
the det Man NOUN []
Man pobj to ADP [the]
" punct influenced VERB []
was auxpass influenced VERB []
influenced ROOT influenced VERB [Power, ", was, by]
by agent influenced VERB [Robben]
the det thinker NOUN []
medioeval compound thinker NOUN []
thinker compound Robben PROPN [the, medioeval]
Mark compound Robben PROPN []
Robben pobj by ADP [thinker, Mark]
(['Power'], ['Robben', 'thinker', 'Mark'])
Jhon's work influenced Mark


CHUNKS
Jhon's work work nsubj influenced
Mark Mark dobj influenced
TREE
Jhon poss work NOUN ['s]
's case Jhon PROPN []
work nsubj influenced VERB [Jhon]
influenced ROOT influenced VERB [work, Mark]
Mark dobj influenced VERB []
([], ['work', 'Mark'])
Marconi's work on the electromagnetic field influenced Einsteins ideas of photon's transmission


CHUNKS
Marconi's work work nsubj influenced
the electromagnetic field field pobj on
Einsteins ideas ideas dobj influenced
photon's transmission transmission pobj of
TREE
Marconi poss work NOUN ['s]
's case Marconi PROPN []
work nsubj influenced VERB [Marconi, on]
on prep work NOUN [field]
the det field NOUN []
electromagnetic amod field NOUN []
field pobj on ADP [the, electromagnetic]
influenced ROOT influenced VERB [work, ideas]
Einsteins compound ideas NOUN []
ideas dobj influenced VERB [Einsteins, of]
of prep ideas NOUN [transmission]
photon poss transmission NOUN ['s]
's case photon PROPN []
transmission pobj of ADP [photon]
([], ['work', 'ideas', 'Einsteins'])
Jhon influenced Mark's work


CHUNKS
Jhon Jhon nsubj influenced
Mark's work work dobj influenced
TREE
Jhon nsubj influenced VERB []
influenced ROOT influenced VERB [Jhon, work]
Mark poss work NOUN ['s]
's case Mark PROPN []
work dobj influenced VERB [Mark]
([], ['Jhon', 'work'])
Pluto influenced the idea of time of Aristotele and Minos


CHUNKS
Pluto Pluto nsubj influenced
the idea idea dobj influenced
time time pobj of
Aristotele Aristotele pobj of
Minos Minos conj Aristotele
TREE
Pluto nsubj influenced VERB []
influenced ROOT influenced VERB [Pluto, idea]
the det idea NOUN []
idea dobj influenced VERB [the, of]
of prep idea NOUN [time]
time pobj of ADP [of]
of prep time NOUN [Aristotele]
Aristotele pobj of ADP [and, Minos]
and cc Aristotele PROPN []
Minos conj Aristotele PROPN []
([], ['Pluto', 'idea'])


#***Import JSON articles***


In [0]:
"""
{
    "philosopher": "name",
    "article": "plaintext_article",
    "pageid": "id",
    "table_influenced": 
        [
            {
                "0": "name_of_someone_philosopher_influenced_by"
            }
        ]
    "table_influences":
        [
            {
                "0": "name_of_someone_philosopher_influences"
            }
        ]
}
"""
#open the file
with open('/content/drive/My Drive/folder/result_1.json') as f:
  jsonlist = json.load(f)

# ***Text Preprocessing***

In [14]:
"""
#HIM
#divide in sentences the article

#eliminate sentences that don't include influenc -e, -ed, -es, -ing etc

#pattern matching on patterns (see language analysis)

#Entity recognition and recostruction

#US
"mark influenced jhon. Today is friday"
#divide in sentences the article
"mark influenced jhon"
"Today is friday"
#eliminate sentences that don't include influenc -e, -ed, -es, etc
"mark influenced jhon"

#pattern matching on patterns (using spacy 
?)
[x = "mark", y = "jhon"]

#Entity recognition and recostruction(here we have a proble, when he had Hendrix he could )
"""

"""
import time
start = time.time()
counter_influence = []
counter_influenced = []
counter_influences = []
counter_influencing = []
counter_inspired = []
counter_inspire = []
counter_inspiration = []
for elem in jsonlist:
    #divide in sentences the article
    sent_list = nltk.sent_tokenize(elem["article"])
    
    sent_list = [word_tokenize(i) for i in sent_list]

    influence_list = []
    influence_declinations = ["influence", "influenced", "influences", "influencing", "inspired", "inspire", "inspiration"]


    for word_list in sent_list:
        temp = [x for x in word_list if x in influence_declinations]
        if "influence" in word_list:
            counter_influence.append(word_list)
        if "influencing" in word_list:
            counter_influencing.append(word_list)
        if "influenced" in word_list:
            counter_influenced.append(word_list)
        if "influences" in word_list:
            counter_influences.append(word_list)
        if "inspired" in word_list:
            counter_inspired.append(word_list)
        if "inspire" in word_list:
            counter_inspire.append(word_list)
        if "inspiration" in word_list:
            counter_inspiration.append(word_list)
        if len(temp) != 0:
            influence_list.append(' '.join(word for word in word_list))

    new_list = []
    for sent in influence_list:
        new_list.append(sent)
        #displacy.render(sent, style="dep", jupyter=True)
    elem["article"] = new_list
print(time.time() - start)

"""
"""
influencing pisciato
inspire pisciato

influence only used as a name
influences only used as a name, plural of influences
inspiration only used as a name, similar to influence
influenced used as verb
inspired used like influenced

X's can be changed with his
Influence:
X be an influence (of/on/for/to) Y 
X's influence (of/on/for/to) Y
X be Y's influence

influences:
influences (include/of) X
X be Y's influences
X and X were influences of Y

influencing:
none

influenced:
Y be influenced by X
X have influenced Y
Y's thinking was influenced by X
X influenced Y
"""




print(len(counter_influence), "influence",
      len(counter_influences), "influences",
      len(counter_influencing), "influencing",
      len(counter_influenced), "influenced",
      len(counter_inspired), "inspired",
      len(counter_inspire), "inspire",
      len(counter_inspiration), "inspiration")
for wl in counter_influence:
    print(' '.join(word for word in wl))

1749 influence 221 influences 35 influencing 1282 influenced 374 inspired 31 inspire 201 inspiration
Chinese Marxist philosophy developed under the influence of Mao Zedong , while a Chinese pragmatism under Hu Shih and New Confucianism 's rise was influenced by Xiong Shili .
authoritarianism An organization or a state that enforces strong , and sometimes oppressive measures against those in its sphere of influence , generally without attempts at gaining their consent and often not allowing feedback on its policies .
It is a complex system of moral , social , political , and religious thought that has had tremendous influence on the history of Chinese civilization down to the 21st century .
historicism The theory that claims that : 1 ) that there is an organic succession of developments ( also known as historism or the German historismus ) , and 2 ) that local conditions and peculiarities influence the results in a decisive way .
Some scholars and anti-Catholic polemicists argue that it

# ***Processing Giuliano(Spacy Dependencies)***

In [0]:

"""
Input cella:
jsonlist = 
[
    {
        "philosopher" = "philosopher_name",
        "pageid" = id,
        "article" = 
        [
            "frase contenente influence",

            "frase contenente influence", etc
        ]
    }
]

Output cella:
output =
[
    {
        "philosopher" = "philosopher_name",
        "pageid" = id,
        "article" = 
        [
            [
                "frase contenente influence",
                [
                    list of influencers
                ],
                [
                    list of influenced
                ]
        ]
    }
]
"""



"""
#some prints and infos

#pprint.pprint(jsonlist)
print(len(jsonlist))
jsonlist_no_empty = [i for i in jsonlist if len(i["article"]) != 0]
print(len(jsonlist_no_empty))
print([[i["philosopher"], len(i["article"]), i["article"]] for i in jsonlist_no_empty if i["philosopher"] == "Socrates"])
"""
"""
import re
print(jsonlist_no_empty[20]["article"][0])
tokens = [i for i in jsonlist_no_empty[20]["article"][0] if i.isupper() ]
print(tokens)
"""

for phil in jsonlist_no_empty:
    display_info(phil["article"])




# ***Processing Mattia(Regex)***

In [0]:

"""
Input cella:
jsonlist = 
[
    {
        "philosopher" = "philosopher_name",
        "pageid" = id,
        "article" = 
        [
            "frase contenente influence",

            "frase contenente influence", etc
        ]
    }
]

Output cella:
output =
[
    {
        "philosopher" = "philosopher_name",
        "pageid" = id,
        "article" = 
        [
            [
                "frase contenente influence",
                [
                    list of influencers
                ],
                [
                    list of influenced
                ]
        ]
    }
]
"""

"""
#some prints and infos

#pprint.pprint(jsonlist)
print(len(jsonlist))
jsonlist_no_empty = [i for i in jsonlist if len(i["article"]) != 0]
print(len(jsonlist_no_empty))
print([[i["philosopher"], len(i["article"]), i["article"]] for i in jsonlist_no_empty if i["philosopher"] == "Socrates"])
"""

# ***Processing Luigi(Regex)***

In [0]:

"""
Input cella:
jsonlist = 
[
    {
        "philosopher" = "philosopher_name",
        "pageid" = id,
        "article" = 
        [
            "frase contenente influence",

            "frase contenente influence", etc
        ]
    }
]

Output cella:
output =
[
    {
        "philosopher" = "philosopher_name",
        "pageid" = id,
        "article" = 
        [
            [
                "frase contenente influence",
                [
                    list of influencers
                ],
                [
                    list of influenced
                ]
        ]
    }
]
"""

"""
#some prints and infos

#pprint.pprint(jsonlist)
print(len(jsonlist))
jsonlist_no_empty = [i for i in jsonlist if len(i["article"]) != 0]
print(len(jsonlist_no_empty))
print([[i["philosopher"], len(i["article"]), i["article"]] for i in jsonlist_no_empty if i["philosopher"] == "Socrates"])
"""